In [1]:
import csv
import os
import nltk
import pickle
# remove numeric, stop words and short words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
import sys
import csv
import pandas as pd
# tokenize the lyrics
from nltk.tokenize import RegexpTokenizer
from gensim.corpora import Dictionary
from gensim.corpora import MmCorpus
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import datetime

csv.field_size_limit(sys.maxsize)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


131072

In [2]:
def get_df(filename):
    # Read the csv file
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)

    df = pd.DataFrame(data)

    df.columns = df.iloc[0]

    df = df.iloc[1:]

    return df

def get_profanities():
    # https://www.freewebheaders.com/youtube-blacklist-words-free-and-youtube-comment-moderation/

    with open('profanity.txt', 'r') as file:
            # split by comma
            profanities = file.read().split(',')
            profanities = [w.strip() for w in profanities]

    return profanities

def remove_profanity(tokens, profanities):

    return [w for w in tokens if not w in profanities]

def get_stopwords():
    stop_words = set(stopwords.words('english'))
    song_specific =  ['lyrics', 'verse']
    pronouns = ['the', 'i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
    pronouns = pronouns + song_specific
    pronouns = set(pronouns)
    stop_words = stop_words.union(pronouns)

    return stop_words

def remove_stopwords(tokens, stop_words):
    return [w for w in tokens if w not in stop_words]

def remove_punctuation(tokens):
    return [w for w in tokens if w not in string.punctuation]

def remove_numeric(tokens):
    return [w for w in tokens if not w.isnumeric()]

def remove_short_words(tokens):
    return [w for w in tokens if len(w) > 2]

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in tokens]

def preprocess_pipeline(tokens,stopwords):
    # lower case
    tokens = [w.lower() for w in tokens]
    tokens = remove_stopwords(tokens,stopwords)
    tokens = remove_punctuation(tokens)
    tokens = remove_numeric(tokens)
    tokens = remove_short_words(tokens)
    tokens = lemmatize(tokens)

    # # remove profanity
    # tokens = remove_profanity(tokens)
    
    return tokens

def tokenize_lyrics(df):

    # tokenise
    tokenizer = RegexpTokenizer(r'\w+')
    stopwords = get_stopwords()

    df['tokens'] = df['Lyrics'].apply(tokenizer.tokenize)

    df["processed_tokens"] = df['tokens'].apply(lambda x : preprocess_pipeline(x,stopwords))

    # remove any song with 0 tokens
    df = df[df['processed_tokens'].map(len) > 0]

    return df

def get_tokens(df):

    # create list of all tokens 
    all_tokens = df['processed_tokens'].tolist()

    return all_tokens

def create_dict(tokens, min_count, max_perc):

    dictionary = Dictionary(tokens)
    dictionary.filter_extremes(no_below = min_count, no_above = max_perc)

    return dictionary

def bow_model(dictionary, all_tokens):

    gensim_corpus = [dictionary.doc2bow(song) for song in all_tokens]
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    return (gensim_corpus, id2word)

def lda_model(params):
    
    lda_model = LdaModel(
    corpus=params["gensim_corpus"],
    id2word= params["id2word"],
    chunksize= params["chunksize"],
    alpha='auto',
    eta='auto',
    iterations=params["iterations"],
    num_topics= params["num_topics"],
    passes= params["passes"]
    )

    return lda_model

def visualise(lda_model,params,dictionary,filename):
    
    vis_data = gensimvis.prepare(lda_model, params["gensim_corpus"], dictionary)
    #pyLDAvis.display(vis_data)
    pyLDAvis.save_html(vis_data, f'./Lyrics_LDA_k_{filename}_'+ str(params["num_topics"]) +'_'+str(params["chunksize"]) + '_final.html')

def bifurcate_df(df, param):

    uniq_vals = list(set(df[param].to_list()))
    ans = []

    for val in uniq_vals:
        df_temp = df[df[param] == val]
        ans.append(df_temp)

    return ans

def get_topic_back(gensim_corpus, lda):

    topics = []
    for doc in gensim_corpus:
        prob = lda[doc]
        topic = max(prob, key=lambda item: item[1])[0]

        # to match topic number of pyldavis
        topics.append(int(topic)+1)
    
    return topics


In [3]:
FILENAME = "filtered_comprehensive.csv"
df = get_df(FILENAME)

# remove any row with lyrics and gender NA
df.dropna(subset=["lower_title", "Artist" , "Lyrics","gender", "year"],inplace=True)

In [4]:
print("Tokenising lyrics")
# get tokens
df = tokenize_lyrics(df)

Tokenising lyrics


In [9]:
_chunksize = 1000
_iterations = 600
_topics =  4

# create bow
gensim_corpus, id2word = bow_model(dictionary, all_tokens)

# create params
params = {
    "gensim_corpus": gensim_corpus,
    "id2word": id2word,
    "chunksize": _chunksize,
    "iterations": _iterations,
    "num_topics": _topics,
    "passes": 20
}

print("Creating LDA model")
# create lda model
lda_model = lda_model(params)

print("Visualising")
# visualise
visualise(lda_model,params,dictionary,"test")

print("Getting Topics...")

topics  = get_topic_back(gensim_corpus, lda_model)

df["topic"] = topics

# Save csv

Creating LDA model
Visualising


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: Deprec

Getting Topics...


In [10]:
df.to_csv(f'topics_'+ str(params["num_topics"]) +'_'+str(params["chunksize"]) + '_final_3Nov.html')
df[['title_x','artist_x','year_x','topic','unique_tokens']].to_csv(f'compact_song_topics_'+ str(params["num_topics"]) +'_'+str(params["chunksize"]) + '_final_3Nov.html')

In [11]:
ans = bifurcate_df(df, 'gender')

In [20]:
# ans[4] is male df
# ans[6] is female df
ans[4].to_csv('male_comprehensive_final_3Nov.csv')
ans[6].to_csv('female_comprehensive_final_3Nov.csv')


In [ ]:
### Navya's code below, realised most work done below a tad bit too late 

In [6]:
df

,song,artist,lyrics
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...
...,...,...,...
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...
758,Good Vibrations,Marky Mark & The Funky Bunch Featuring Loleatt...,
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...


In [7]:
# tokenize the lyrics
from nltk.tokenize import RegexpTokenizer

In [8]:
# tokenise
tokenizer = RegexpTokenizer(r'\w+')

df['tokens'] = df['lyrics'].apply(tokenizer.tokenize)

In [9]:
df

,song,artist,lyrics,tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold..."
...,...,...,...,...
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...,"[Emotions, Lyrics, Chorus, You, ve, got, me, f..."
758,Good Vibrations,Marky Mark & The Funky Bunch Featuring Loleatt...,,[]
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ...","[I, Adore, Mi, Amor, LyricsDream, on, dream, a..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...,"[I, Don, t, Wanna, Cry, Lyrics, Verse, 1, Once..."


In [11]:
# remove numeric, stop words and short words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

stop_words = set(stopwords.words('english'))

# add pronouns to stop words

# list of pronouns

song_specific =  ['lyrics', 'verse']

pronouns = ['the', 'i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

pronouns = pronouns + song_specific

pronouns = set(pronouns)

stop_words = stop_words.union(pronouns)

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# remove profanities?

# https://www.freewebheaders.com/youtube-blacklist-words-free-and-youtube-comment-moderation/

with open('profanity.txt', 'r') as file:
    # split by comma
    profanities = file.read().split(',')
    profanities = [w.strip() for w in profanities]

def remove_stopwords(tokens):
    return [w for w in tokens if w not in stop_words]

def remove_punctuation(tokens):
    return [w for w in tokens if not w in string.punctuation]

def remove_numeric(tokens):
    return [w for w in tokens if not w.isnumeric()]

def remove_short_words(tokens):
    return [w for w in tokens if len(w) > 2]

def lemmatize(tokens):
    return [lemmatizer.lemmatize(w) for w in tokens]

def remove_profanity(tokens):
    return [w for w in tokens if not w in profanities]


In [12]:
def preprocess_pipeline(tokens):
    # lower case
    tokens = [w.lower() for w in tokens]
    tokens = remove_stopwords(tokens)
    tokens = remove_punctuation(tokens)
    tokens = remove_numeric(tokens)
    tokens = remove_short_words(tokens)
    tokens = lemmatize(tokens)

    # remove profanity
    tokens = remove_profanity(tokens)
    
    return tokens

In [13]:
df["processed_tokens"]  = df["tokens"].apply(preprocess_pipeline)

In [14]:
df

,song,artist,lyrics,tokens,processed_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,...","[another, day, paradise, call, man, street, si..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,...","[baby, forget, number, lyricsokay, okay, eddie..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ...","[rap, collection, lyricsa, hempstead, high, lo..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y...","[blame, rain, said, need, told, goodbye, goodb..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold...","[cold, hearted, chorus, cold, hearted, snake, ..."
...,...,...,...,...,...
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...,"[Emotions, Lyrics, Chorus, You, ve, got, me, f...","[emotion, chorus, got, feeling, emotion, deepe..."
758,Good Vibrations,Marky Mark & The Funky Bunch Featuring Loleatt...,,[],[]
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ...","[I, Adore, Mi, Amor, LyricsDream, on, dream, a...","[adore, amor, lyricsdream, dream, away, think,..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...,"[I, Don, t, Wanna, Cry, Lyrics, Verse, 1, Once...","[wanna, cry, sit, silence, said, done, emptine..."


In [15]:
# remove any song with 0 tokens
df = df[df['processed_tokens'].map(len) > 0]

In [21]:
# create list of all tokens 
all_tokens = df['processed_tokens'].tolist()

In [22]:
from gensim.corpora import Dictionary
dictionary = Dictionary(all_tokens)
dictionary.filter_extremes(no_below = 20, no_above = 0.8)

In [23]:
from gensim.corpora import MmCorpus
gensim_corpus = [dictionary.doc2bow(song) for song in all_tokens]
temp = dictionary[0]
id2word = dictionary.id2token

In [34]:
chunksize = 2000
passes = 20
iterations = 400
num_topics = 4

In [35]:
from gensim.models import LdaModel
lda_model = LdaModel(
corpus=gensim_corpus,
id2word=id2word,
chunksize=chunksize,
alpha='auto',
eta='auto',
iterations=iterations,
num_topics=num_topics,
passes=passes
)

In [36]:
from gensim.models.coherencemodel import CoherenceModel
coherencemodel = CoherenceModel(model=lda_model, texts=all_tokens, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

0.4706439045829762


In [37]:
df

,song,artist,lyrics,tokens,processed_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,...","[another, day, paradise, call, man, street, si..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,...","[baby, forget, number, lyricsokay, okay, eddie..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ...","[rap, collection, lyricsa, hempstead, high, lo..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y...","[blame, rain, said, need, told, goodbye, goodb..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold...","[cold, hearted, chorus, cold, hearted, snake, ..."
...,...,...,...,...,...
755,Coming Out Of The Dark,Gloria Estefan,Coming Out of the Dark Lyrics[Verse 1]\nWhy be...,"[Coming, Out, of, the, Dark, Lyrics, Verse, 1,...","[coming, dark, afraid, alone, though, life, ne..."
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...,"[Emotions, Lyrics, Chorus, You, ve, got, me, f...","[emotion, chorus, got, feeling, emotion, deepe..."
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ...","[I, Adore, Mi, Amor, LyricsDream, on, dream, a...","[adore, amor, lyricsdream, dream, away, think,..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...,"[I, Don, t, Wanna, Cry, Lyrics, Verse, 1, Once...","[wanna, cry, sit, silence, said, done, emptine..."


In [38]:
!pip install pyLDAvis

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
#pyLDAvis.display(vis_data)
pyLDAvis.save_html(vis_data, './Lyrics_LDA_k_'+ str(num_topics) +'.html')

/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anac

In [ ]:
#####